In [23]:
# ==========================================
# CELL 1: Install Required Packages
# ==========================================
!pip install transformers torch pandas accelerate bitsandbytes

In [1]:
# ==========================================
# CELL 2: Import Libraries
# ==========================================
import pandas as pd
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from typing import Dict, List, Any
import logging
import gc

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
# ==========================================
# CELL 3: Define ThaiLegalGuardrail Class
# ==========================================
class ThaiLegalGuardrail:
    _instance = None
    _model = None
    _tokenizer = None
    _model_loaded = False

    def __new__(cls, model_name: str = "scb10x/llama3.1-typhoon2-8b-instruct"):
        """Singleton pattern to ensure only one model instance"""
        if cls._instance is None:
            cls._instance = super(ThaiLegalGuardrail, cls).__new__(cls)
        return cls._instance

    def __init__(self, model_name: str = "scb10x/llama3.1-typhoon2-8b-instruct"):
        """Initialize the Thai Legal Guardrail System"""
        if not hasattr(self, 'initialized'):
            self.model_name = model_name
            self.civil_commercial_keywords = self._get_civil_commercial_keywords()
            self.other_law_keywords = self._get_other_law_keywords()
            self.initialized = True

    def _get_civil_commercial_keywords(self) -> List[str]:
        """Keywords related to Civil and Commercial Code"""
        return [
            "ประมวลกฎหมายแพ่งและพาณิชย์", "แพ่งและพาณิชย์", "ประมวลแพ่งและพาณิชย์",
            "สัญญา", "ซื้อขาย", "เช่า", "กู้ยืม", "จำนอง", "จำนำ", "รับเหมา",
            "หุ้นส่วน", "บริษัท", "ทรัพย์สิน", "มรดก", "หนี้", "เจ้าหนี้", "ลูกหนี้",
            "ค่าเสียหาย", "ผิดสัญญา", "เลิกสัญญา", "บอกเลิกสัญญา", "การค้า",
            "พาณิชย์", "ทุนจดทะเบียน", "หุ้น", "ผู้ถือหุ้น", "กรรมการ",
            "ห้างหุ้นส่วน", "บริษัทจำกัด", "บริษัทมหาชน", "นิติบุคคล"
        ]

    def _get_other_law_keywords(self) -> List[str]:
        """Keywords for other types of law (should be filtered out)"""
        return [
            "อาญา", "ประมวลกฎหมายอาญา", "ฆ่า", "ข่มขืน", "ลักทรัพย์", "โจรกรรม",
            "ยาเสพติด", "รัฐธรรมนูญ", "การปกครอง", "ภาษี", "ศุลกากร",
            "แรงงาน", "ประกันสังคม", "สิ่งแวดล้อม", "ป่าไผ่", "ประมง",
            "คุ้มครองผู้บริโภค", "ข้อมูลข่าวสาร", "การศึกษา", "สาธารณสุข",
            "หลักทรัพย์", "สัญญาซื้อขายล่วงหน้า", "กองทุน", "ประกันภัย",
            "ธนาคาร", "สถาบันการเงิน"
        ]

    def load_model(self):
        """Load the model and tokenizer only once"""
        if ThaiLegalGuardrail._model_loaded:
            logger.info("Model already loaded, skipping...")
            return

        try:
            logger.info(f"Loading model: {self.model_name}")

            # Clear GPU memory first
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gc.collect()

            ThaiLegalGuardrail._tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            ThaiLegalGuardrail._model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )

            # Add pad token if it doesn't exist
            if ThaiLegalGuardrail._tokenizer.pad_token is None:
                ThaiLegalGuardrail._tokenizer.pad_token = ThaiLegalGuardrail._tokenizer.eos_token

            ThaiLegalGuardrail._model_loaded = True
            logger.info("Model loaded successfully")

        except Exception as e:
            logger.error(f"Error loading model: {e}")
            raise

    @property
    def model(self):
        """Get the loaded model"""
        if not ThaiLegalGuardrail._model_loaded:
            self.load_model()
        return ThaiLegalGuardrail._model

    @property
    def tokenizer(self):
        """Get the loaded tokenizer"""
        if not ThaiLegalGuardrail._model_loaded:
            self.load_model()
        return ThaiLegalGuardrail._tokenizer

    def layer1_keyword_filter(self, text: str) -> Dict[str, Any]:
        """First layer: Filter out keywords not related to Civil and Commercial Code"""
        text_lower = text.lower()

        # Check for other law keywords (should be filtered out)
        other_law_found = any(keyword in text_lower for keyword in self.other_law_keywords)

        # Check for civil/commercial keywords
        civil_commercial_found = any(keyword in text_lower for keyword in self.civil_commercial_keywords)

        if other_law_found and not civil_commercial_found:
            return {
                "passed": False,
                "reason": "Contains keywords related to other types of law",
                "decision": "not allowed"
            }

        if not civil_commercial_found:
            return {
                "passed": False,
                "reason": "No Civil and Commercial Code related keywords found",
                "decision": "not allowed"
            }

        return {
            "passed": True,
            "reason": "Contains Civil and Commercial Code related keywords",
            "decision": "potential_allowed"
        }

    def _generate_response(self, prompt: str) -> str: # Removed max_length from signature
        """Generate response using the loaded model"""
        try:
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True)

            # Move inputs to the same device as model
            if torch.cuda.is_available():
                inputs = {k: v.cuda() for k, v in inputs.items()}

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=200, # Use max_new_tokens to control output length
                    max_length=1024,
                    temperature=0.1,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id
                )

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            # Remove the input prompt from response
            response = response[len(prompt):].strip()
            return response

        except Exception as e:
            logger.error(f"Error generating response: {e}")
            return ""

    def layer2_input_checking(self, user_input: str) -> Dict[str, Any]:
        """Second layer: LLM input checking to decide if it's related to Civil and Commercial Code"""
        system_prompt = """You are an expert Thai legal classifier. Your task is to determine if a question is related to Thai Civil and Commercial Code (ประมวลกฎหมายแพ่งและพาณิชย์) or not.

Civil and Commercial Code covers:
- Contracts (สัญญา)
- Property rights (สิทธิในทรัพย์)
- Obligations and debts (หนี้และภาระ)
- Commercial law (กฎหมายพาณิชย์)
- Company law (กฎหมายบริษัท)
- Partnership (ห้างหุ้นส่วน)
- Sale and purchase (ซื้อขาย)
- Lease (เช่า)
- Loan (กู้ยืม)
- Mortgage (จำนอง)
- Inheritance (มรดก)

NOT Civil and Commercial Code:
- Criminal law (กฎหมายอาญา)
- Administrative law (กฎหมายปกครอง)
- Securities law (กฎหมายหลักทรัพย์)
- Banking law (กฎหมายธนาคาร)
- Insurance law (กฎหมายประกันภัย)
- Tax law (กฎหมายภาษี)

Respond with only "YES" if the question is related to Civil and Commercial Code, or "NO" if it's related to other types of law or non-legal topics."""

        prompt = f"{system_prompt}\n\nQuestion: {user_input}\n\nAnswer:"

        response = self._generate_response(prompt) # Removed max_length from here

        # Extract YES/NO from response
        response_clean = response.upper().strip()

        if "YES" in response_clean:
            return {
                "passed": True,
                "reason": "Input is related to Civil and Commercial Code",
                "decision": "allowed"
            }
        else:
            return {
                "passed": False,
                "reason": "Input is not related to Civil and Commercial Code",
                "decision": "not allowed"
            }

    def layer3_output_checking(self, output: str, original_input: str) -> Dict[str, Any]:
        """Third layer: LLM output checking if output is related to Civil and Commercial Code"""
        system_prompt = """You are an expert Thai legal classifier. Your task is to determine if a legal answer/output is related to Thai Civil and Commercial Code (ประมวลกฎหมายแพ่งและพาณิชย์) or other types of law.

Analyze the legal content and determine if it discusses Civil and Commercial Code matters or other legal areas like criminal law, administrative law, tax law, etc.

Respond with only "YES" if the output discusses Civil and Commercial Code matters, or "NO" if it discusses other types of law."""

        prompt = f"{system_prompt}\n\nOriginal Question: {original_input}\n\nLegal Answer: {output}\n\nIs this related to Civil and Commercial Code? Answer:"

        response = self._generate_response(prompt) # Removed max_length from here

        # Extract YES/NO from response
        response_clean = response.upper().strip()

        if "YES" in response_clean:
            return {
                "passed": True,
                "reason": "Output is related to Civil and Commercial Code",
                "decision": "allowed"
            }
        else:
            return {
                "passed": False,
                "reason": "Output is not related to Civil and Commercial Code",
                "decision": "not allowed"
            }

    def full_guardrail_check(self, user_input: str, model_output: str = None) -> Dict[str, Any]:
        """Run all three layers of guardrail checking"""
        # Layer 1: Keyword filtering
        layer1_result = self.layer1_keyword_filter(user_input)
        if not layer1_result["passed"]:
            return {
                "final_decision": "not allowed",
                "reason": layer1_result["reason"],
                "layer_stopped": 1,
                "layer1": layer1_result
            }

        # Layer 2: Input checking with LLM
        layer2_result = self.layer2_input_checking(user_input)
        if not layer2_result["passed"]:
            return {
                "final_decision": "not allowed",
                "reason": layer2_result["reason"],
                "layer_stopped": 2,
                "layer1": layer1_result,
                "layer2": layer2_result
            }

        # Layer 3: Output checking (if output is provided)
        if model_output:
            layer3_result = self.layer3_output_checking(model_output, user_input)
            return {
                "final_decision": layer3_result["decision"],
                "reason": layer3_result["reason"],
                "layer_stopped": 3,
                "layer1": layer1_result,
                "layer2": layer2_result,
                "layer3": layer3_result
            }

        # If no output to check, return layer 2 result
        return {
            "final_decision": layer2_result["decision"],
            "reason": layer2_result["reason"],
            "layer_stopped": 2,
            "layer1": layer1_result,
            "layer2": layer2_result
        }

In [3]:
# ==========================================
# CELL 4: Data Loading and Transformation Functions
# ==========================================
import ast # Import the ast module
import json # Keep json import in case needed elsewhere

def load_and_transform_data(csv_path: str) -> pd.DataFrame:
    """Load and transform the CCL dataset for testing"""
    try:
        df = pd.read_csv(csv_path)
        logger.info(f"Loaded {len(df)} records from {csv_path}")

        # Transform the data for testing
        test_data = []

        for idx, row in df.iterrows():
            question = row['question']
            relevant_laws_str = row['relevant_laws']

            relevant_laws = []
            is_civil_commercial = False
            law_names = []

            # Attempt to parse the string as a Python literal (handles single quotes)
            try:
                parsed_laws = ast.literal_eval(relevant_laws_str)
                if isinstance(parsed_laws, list):
                    relevant_laws = parsed_laws
                else:
                     logger.warning(f"Row {idx}: Parsed data is not a list: {relevant_laws_str}")


            except (ValueError, SyntaxError) as e: # Catch Value Error and SyntaxError for ast.literal_eval
                logger.warning(f"Row {idx}: Failed to parse literal: {relevant_laws_str} - {e}")
                continue # Skip this row if parsing fails


            # Process the parsed relevant_laws list
            for law in relevant_laws:
                if isinstance(law, dict):
                    law_name = law.get('law_name', '')
                    law_names.append(law_name)

                    # Check if it contains Civil and Commercial Code keywords
                    if any(keyword in law_name for keyword in [
                        "ประมวลกฎหมายแพ่งและพาณิชย์",
                        "แพ่งและพาณิชย์",
                        "Civil and Commercial Code"
                    ]):
                        is_civil_commercial = True
                        break
                else:
                     logger.warning(f"Row {idx}: Item in relevant_laws is not a dictionary: {law}")


            expected_decision = "allowed" if is_civil_commercial else "not allowed"

            test_data.append({
                'question': question,
                'relevant_laws': relevant_laws,
                'law_names': law_names,
                'is_civil_commercial': is_civil_commercial,
                'expected_decision': expected_decision,
                'original_answer': row.get('answer', ''),
            })


        test_df = pd.DataFrame(test_data)
        logger.info(f"Transformed {len(test_df)} records successfully")

        return test_df

    except Exception as e:
        logger.error(f"Error loading data: {e}")
        raise

In [4]:
# ==========================================
# CELL 5: Initialize and Load Model (Run Once!)
# ==========================================
# Initialize the guardrail system
logger.info("Initializing Thai Legal Guardrail System...")
guardrail = ThaiLegalGuardrail()

# Load the model (this will only happen once due to singleton pattern)
guardrail.load_model()

print("Model loaded successfully! Ready for testing.")





/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully! Ready for testing.


In [5]:
# ==========================================
# CELL 6: Test Single Example
# ==========================================
def test_single_example(input_text: str):
    """Test with a single example"""
    result = guardrail.full_guardrail_check(input_text)

    print(f"Input: {input_text}")
    print(f"Decision: {result['final_decision']}")
    print(f"Reason: {result['reason']}")
    print(f"Layer stopped: {result['layer_stopped']}")

    return result

# Test with the provided example
example_input = "ถ้ามีคนประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตต้องระวางโทษอย่างไร"
test_single_example(example_input)

Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Input: ถ้ามีคนประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตต้องระวางโทษอย่างไร
Decision: allowed
Reason: Input is related to Civil and Commercial Code
Layer stopped: 2


{'final_decision': 'allowed',
 'reason': 'Input is related to Civil and Commercial Code',
 'layer_stopped': 2,
 'layer1': {'passed': True,
  'reason': 'Contains Civil and Commercial Code related keywords',
  'decision': 'potential_allowed'},
 'layer2': {'passed': True,
  'reason': 'Input is related to Civil and Commercial Code',
  'decision': 'allowed'}}

In [6]:
# ==========================================
# CELL 7: Load and Preview Test Data
# ==========================================
# Load and transform test data
csv_path = "/content/ccl.csv"  # Make sure to upload this file to Colab
logger.info("Loading and transforming test data...")
test_df = load_and_transform_data(csv_path)

# Display data distribution
print(f"Data distribution:")
print(f"Total samples: {len(test_df)}")
print(f"Civil & Commercial: {sum(test_df['is_civil_commercial'])}")
print(f"Other laws: {sum(~test_df['is_civil_commercial'])}")

# Preview first few samples
print("\nFirst 5 samples:")
for i in range(min(5, len(test_df))):
    row = test_df.iloc[i]
    print(f"\nSample {i+1}:")
    print(f"Question: {row['question'][:100]}...")
    print(f"Expected: {row['expected_decision']}")
    print(f"Laws: {row['law_names']}")

Data distribution:
Total samples: 3729
Civil & Commercial: 1617
Other laws: 2112

First 5 samples:

Sample 1:
Question: ถ้ามีคนประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตต้องระวางโทษอย่าง...
Expected: not allowed
Laws: ['พระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546']

Sample 2:
Question: ถ้าผู้ิยู่ในปกครองได้ยินยอมในการกระทำของผู้ปกครองจะทำให้ผู้ปกครองหลุดพ้นจากความรับผิดหรือเปล่า...
Expected: allowed
Laws: ['ประมวลกฎหมายแพ่งและพาณิชย์']

Sample 3:
Question: ถ้าผู้รับประกันภัยต้องคำพิพากษาให้เป็นคนล้มละลาย ผู้เอาประกันภัยต้องทำอย่างไร...
Expected: allowed
Laws: ['ประมวลกฎหมายแพ่งและพาณิชย์']

Sample 4:
Question: ถ้าผู้เป็นหุ้นส่วนให้ความเป็นเจ้าของในทรัพย์สินอันใดอันหนึ่งเป็นการลงหุ้นแล้วทรัพย์สินเกิดชำรุดบกพร่...
Expected: allowed
Laws: ['ประมวลกฎหมายแพ่งและพาณิชย์']

Sample 5:
Question: ถ้ามีการรับชำระหนี้ตามสัญญา นายหน้ามีอำนาจจะรับชำระหนี้ไว้หรือไม่...
Expected: allowed
Laws: ['ประมวลกฎหมายแพ่งและพาณิชย์']


In [7]:
# ==========================================
# CELL 8: Evaluation Functions
# ==========================================
def run_evaluation(test_df: pd.DataFrame, sample_size: int = None) -> Dict[str, float]:
    """Run evaluation on the test dataset"""

    # Use sample if specified
    if sample_size and sample_size < len(test_df):
        test_df = test_df.sample(n=sample_size, random_state=42)
        logger.info(f"Using sample of {sample_size} records for evaluation")

    results = []
    correct_predictions = 0
    total_predictions = len(test_df)

    # Initialize confusion matrix components
    tp = 0 # True Positives (Allowed correctly predicted as Allowed)
    fn = 0 # False Negatives (Allowed incorrectly predicted as Not Allowed)
    tn = 0 # True Negatives (Not Allowed correctly predicted as Not Allowed)
    fp = 0 # False Positives (Not Allowed incorrectly predicted as Allowed)


    logger.info(f"Starting evaluation on {total_predictions} samples...")

    for idx, row in test_df.iterrows():
        if idx % 10 == 0:
            logger.info(f"Processing sample {len(results)+1}/{total_predictions}")

        question = row['question']
        expected_decision = row['expected_decision']

        # Run guardrail check
        try:
            result = guardrail.full_guardrail_check(question)
            predicted_decision = result['final_decision']

            is_correct = predicted_decision == expected_decision
            if is_correct:
                correct_predictions += 1

            # Update confusion matrix
            if expected_decision == 'allowed' and predicted_decision == 'allowed':
                tp += 1
            elif expected_decision == 'allowed' and predicted_decision == 'not allowed':
                fn += 1
            elif expected_decision == 'not allowed' and predicted_decision == 'not allowed':
                tn += 1
            elif expected_decision == 'not allowed' and predicted_decision == 'allowed':
                fp += 1


            results.append({
                'question': question,
                'expected_decision': expected_decision,
                'predicted_decision': predicted_decision,
                'is_correct': is_correct,
                'reason': result['reason'],
                'layer_stopped': result['layer_stopped'],
                'law_names': row['law_names']
            })

        except Exception as e:
            logger.error(f"Error processing sample {idx}: {e}")
            results.append({
                'question': question,
                'expected_decision': expected_decision,
                'predicted_decision': 'error',
                'is_correct': False,
                'reason': f'Error: {str(e)}',
                'layer_stopped': 0,
                'law_names': row['law_names']
            })

    accuracy = correct_predictions / total_predictions

    # Calculate metrics
    # allowed_correct = sum(1 for r in results if r['expected_decision'] == 'allowed' and r['is_correct'])
    # allowed_total = sum(1 for r in results if r['expected_decision'] == 'allowed')
    # not_allowed_correct = sum(1 for r in results if r['expected_decision'] == 'not allowed' and r['is_correct'])
    # not_allowed_total = sum(1 for r in results if r['expected_decision'] == 'not allowed')

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0


    metrics = {
        'overall_accuracy': accuracy,
        'total_samples': total_predictions,
        'correct_predictions': correct_predictions,
        # 'allowed_accuracy': allowed_correct / allowed_total if allowed_total > 0 else 0, # Kept for comparison
        # 'not_allowed_accuracy': not_allowed_correct / not_allowed_total if not_allowed_total > 0 else 0, # Kept for comparison
        'allowed_total': tp + fn, # Total actual Allowed
        'not_allowed_total': tn + fp, # Total actual Not Allowed
        'true_positives': tp,
        'false_negatives': fn,
        'true_negatives': tn,
        'false_positives': fp,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

    return metrics, results

def print_evaluation_results(metrics: Dict[str, float], detailed_results: List[Dict]):
    """Print evaluation results in a formatted way"""
    print("\n" + "="*50)
    print("EVALUATION RESULTS")
    print("="*50)
    print(f"Overall Accuracy: {metrics['overall_accuracy']:.3f}")
    print(f"Total Samples: {metrics['total_samples']}")
    print(f"Correct Predictions: {metrics['correct_predictions']}")
    # print(f"Allowed Accuracy: {metrics['allowed_accuracy']:.3f} ({metrics['allowed_total']} samples)") # Kept for comparison
    # print(f"Not Allowed Accuracy: {metrics['not_allowed_accuracy']:.3f} ({metrics['not_allowed_total']} samples)") # Kept for comparison

    print("\nConfusion Matrix:")
    print(f"  Actual Allowed (Total: {metrics['allowed_total']})")
    print(f"    Predicted Allowed: {metrics['true_positives']} (True Positives)")
    print(f"    Predicted Not Allowed: {metrics['false_negatives']} (False Negatives)")
    print(f"  Actual Not Allowed (Total: {metrics['not_allowed_total']})")
    print(f"    Predicted Not Allowed: {metrics['true_negatives']} (True Negatives)")
    print(f"    Predicted Allowed: {metrics['false_positives']} (False Positives)")

    print("\nClassification Metrics (for 'Allowed' class):")
    print(f"Precision: {metrics['precision']:.3f}")
    print(f"Recall: {metrics['recall']:.3f}")
    print(f"F1 Score: {metrics['f1_score']:.3f}")


    # Show some example results
    print("\n" + "-"*50)
    print("EXAMPLE RESULTS")
    print("-"*50)

    for i, result in enumerate(detailed_results[:5]):
        print(f"\nExample {i+1}:")
        print(f"Question: {result['question'][:100]}...")
        print(f"Expected: {result['expected_decision']}")
        print(f"Predicted: {result['predicted_decision']}")
        print(f"Correct: {result['is_correct']}")
        print(f"Reason: {result['reason']}")

In [8]:
# ==========================================
# CELL 9: Run Small Sample Evaluation (Optional)
# ==========================================
# Test with a small sample first
print("Running evaluation on 200 samples...")
metrics_sample, results_sample = run_evaluation(test_df, sample_size=200)
print_evaluation_results(metrics_sample, results_sample)





Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Running evaluation on 200 samples...


Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati


EVALUATION RESULTS
Overall Accuracy: 0.625
Total Samples: 200
Correct Predictions: 125

Confusion Matrix:
  Actual Allowed (Total: 103)
    Predicted Allowed: 51 (True Positives)
    Predicted Not Allowed: 52 (False Negatives)
  Actual Not Allowed (Total: 97)
    Predicted Not Allowed: 74 (True Negatives)
    Predicted Allowed: 23 (False Positives)

Classification Metrics (for 'Allowed' class):
Precision: 0.689
Recall: 0.495
F1 Score: 0.576

--------------------------------------------------
EXAMPLE RESULTS
--------------------------------------------------

Example 1:
Question: คำบอกกล่าวขาดความเชื่อถือในกรณีไม่ใช้เงินนั้น จะต้องระบุรายการลงไปอะไรบ้าง...
Expected: allowed
Predicted: not allowed
Correct: False
Reason: No Civil and Commercial Code related keywords found

Example 2:
Question: ใบเสร็จรับเงินที่กรมศุลกากรหรือกรมสรรพสามิตออกให้ในการเรียกเก็บภาษีมูลค่าเพิ่มเพื่อกรมสรรพากรสำหรับส...
Expected: not allowed
Predicted: not allowed
Correct: True
Reason: Contains keywords related 

In [ ]:
# ==========================================
# CELL 10: Run Full Evaluation (Optional - Run this if you want full results)
# ==========================================
# Uncomment the lines below to run full evaluation
print("Running full evaluation...")
metrics_full, results_full = run_evaluation(test_df)
print_evaluation_results(metrics_full, results_full)

# # Save detailed results
results_df = pd.DataFrame(results_full)
results_df.to_csv('guardrail_evaluation_results.csv', index=False)
logger.info("Detailed results saved to 'guardrail_evaluation_results.csv'")

Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Running full evaluation...


Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati

In [ ]:
# ==========================================
# CELL 11: Interactive Testing
# ==========================================
def interactive_test():
    """Interactive testing function"""
    print("Interactive Thai Legal Guardrail Testing")
    print("Type 'quit' to exit")

    while True:
        user_input = input("\nEnter your legal question (Thai): ")
        if user_input.lower() == 'quit':
            break

        result = guardrail.full_guardrail_check(user_input)

        print(f"\nDecision: {result['final_decision']}")
        print(f"Reason: {result['reason']}")
        print(f"Layer stopped: {result['layer_stopped']}")

        # Show layer details
        for layer_num in range(1, result['layer_stopped'] + 1):
            layer_key = f"layer{layer_num}"
            if layer_key in result:
                print(f"Layer {layer_num}: {result[layer_key]['reason']}")

# Uncomment to run interactive testing
interactive_test()